<a href="https://colab.research.google.com/github/ilinashah177/Coursework-Website/blob/main/CO2stepwise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install statsmodels pandas numpy

import pandas as pd
import numpy as np
import statsmodels.api as sm

In [14]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving CO2Step.xlsx to CO2Step.xlsx


In [16]:
import pandas as pd

# Name of uploaded file
file_name = 'CO2Step.xlsx'

# Read data from the required sheet
df = pd.read_excel(file_name, sheet_name='Sheet1', engine='openpyxl') # Pass file_name and specify sheet_name

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df

,Year,BRI Investments (USD millions),Total Population,Urbanization (% of urban population),GDP (USD Billion),Annual CO2 emissions (millon tonnes)
0,2015,520.40,199.427,36.03,299.96,150.59
1,2016,893.25,203.631,36.23,313.63,178.03
2,2017,2296.00,207.906,36.44,339.21,218.36
3,2018,2062.00,212.228,36.67,356.13,210.35
4,2019,2907.00,216.565,36.91,320.91,207.62
5,2020,1600.00,220.892,37.16,300.43,230.75
6,2021,2400.00,225.199,37.44,348.52,247.90
7,2022,2716.00,235.824,37.73,374.79,223.83
8,2023,1440.00,240.486,38.04,338.37,200.67


In [17]:
#Define dependent variable (environmental indicator, e.g., CO2 emissions)
y = df['Annual CO2 emissions (millon tonnes)']  # This is your dependent variable

#Define the independent variables (explanatory variables)
X = df[['BRI Investments (USD millions)', 'Total Population', 'Urbanization (% of urban population)', 'GDP (USD Billion)']]

#Add a constant term (intercept) to the model
X = sm.add_constant(X)

In [18]:
#Function for backward elimination
def backward_elimination(X, y, significance_level=0.05):
    # Start with all predictors
    X_selected = X.copy()

    while True:
        # Fit the model with the current predictors
        model = sm.OLS(y, X_selected).fit()

        # Get the p-values of the variables
        p_values = model.pvalues[1:]  # Exclude intercept

        # Check for the highest p-value
        max_p_value = p_values.max()

        # If the highest p-value is greater than the significance level, remove the variable
        if max_p_value > significance_level:
            excluded_var = p_values.idxmax()
            X_selected = X_selected.drop(columns=[excluded_var])
            print(f"Removed variable: {excluded_var} with p-value: {max_p_value:.4f}")
        else:
            # If all variables have p-values below the significance level, break the loop
            break

    return model

In [19]:
# Perform backward elimination
final_model = backward_elimination(X, y)

# Print the summary of the final model
print("\nFinal Model Summary:")
print(final_model.summary())

Removed variable: GDP (USD Billion) with p-value: 0.5674
Removed variable: Total Population with p-value: 0.1225
Removed variable: Urbanization (% of urban population) with p-value: 0.2524

Final Model Summary:
                                     OLS Regression Results                                     
Dep. Variable:     Annual CO2 emissions (millon tonnes)   R-squared:                       0.570
Model:                                              OLS   Adj. R-squared:                  0.508
Method:                                   Least Squares   F-statistic:                     9.274
Date:                                  Fri, 24 Jan 2025   Prob (F-statistic):             0.0187
Time:                                          16:02:19   Log-Likelihood:                -38.744
No. Observations:                                     9   AIC:                             81.49
Df Residuals:                                         7   BIC:                             81.88
Df Model:    

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  res = hypotest_fun_out(*samples, **kwds)


In [20]:
#Function to perform Stepwise Regression
def stepwise_regression(X, y, direction="both", threshold_in=0.05, threshold_out=0.10):
    """
    Performs stepwise regression using both forward and backward elimination.
    """
    initial_list = X.columns.tolist()
    while True:
        changed = False
        #Forward Step: Adding variables to the model
        if direction in ["both", "forward"]:
            model = sm.OLS(y, X[initial_list]).fit()
            p_values = model.pvalues[1:]  # Exclude intercept
            max_p_value = p_values.max()
            if max_p_value > threshold_in:
                excluded_var = p_values.idxmax()
                initial_list.remove(excluded_var)
                changed = True

        #Backward Step: Removing variables from the model
        if direction in ["both", "backward"]:
            model = sm.OLS(y, X[initial_list]).fit()
            p_values = model.pvalues[1:]  # Exclude intercept
            max_p_value = p_values.max()
            if max_p_value > threshold_out:
                excluded_var = p_values.idxmax()
                initial_list.remove(excluded_var)
                changed = True

        # If no change, break the loop
        if not changed:
            break

    # Final model with selected variables
    final_model = sm.OLS(y, X[initial_list]).fit()
    return final_model

# Perform stepwise regression
stepwise_model = stepwise_regression(X, y)

# Print the summary of the final model
print(stepwise_model.summary())

                                     OLS Regression Results                                     
Dep. Variable:     Annual CO2 emissions (millon tonnes)   R-squared:                       0.570
Model:                                              OLS   Adj. R-squared:                  0.508
Method:                                   Least Squares   F-statistic:                     9.274
Date:                                  Fri, 24 Jan 2025   Prob (F-statistic):             0.0187
Time:                                          16:03:06   Log-Likelihood:                -38.744
No. Observations:                                     9   AIC:                             81.49
Df Residuals:                                         7   BIC:                             81.88
Df Model:                                             1                                         
Covariance Type:                              nonrobust                                         
                              

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  res = hypotest_fun_out(*samples, **kwds)
